In [10]:
import os
import glob
import re

from moviepy.editor import VideoFileClip, concatenate_videoclips, CompositeVideoClip, TextClip, ImageClip, afx
import cairosvg
import html
from common import get_videos


video_length = 20
muted_length = 5
intro_length = 2
cross_fade = 2
temp_dir = 'svg_temp'
width = 1080
height = 1920

def get_video_meta():
    videos = []
    with open('list.csv', 'r', encoding='utf-8') as f:
        reader = csv.reader(f, dialect='mydialect')
        for row in reader:
            if len(row) > 0:
                videos.append({'year': int(row[0]), 'ranking': int(row[1]), 'artist': row[2], 'song': row[3]})
    return videos

def get_songs(year, start, end):
    videos = get_videos(year, start, end)
    for video in videos:
        if video['status'] != 'OK':
            print(f'Video {video["ranking"]} {video["artist"]} {video["title"]} not found: {video["status"]}')
    return videos

def make_missing():
    svg = f"""<?xml version="1.0" encoding="UTF-8"?>
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="1080" height="1920">
    <text font-family="Russo One" font-size="70" fill="white" stroke="black" stroke-width="3" text-anchor="middle" x="50%" y="50%">
        <tspan x="50%" dy="1em">Ääniraitaa ei löytynyt :(</tspan>
    </text>
</svg>
    """
    filename = f"{temp_dir}/missing.png"
    cairosvg.svg2png(bytestring=svg.encode(), write_to=filename, output_width=width, output_height=height)
    return ImageClip(filename)

def make_muted():
    svg = f"""<?xml version="1.0" encoding="UTF-8"?>
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="1080" height="1920">
    <text font-family="Russo One" font-size="60" fill="white" stroke="black" stroke-width="3" text-anchor="middle" x="50%" y="50%">
        <tspan x="50%" dy="1em">Ääni poistettu tekijänoikeussyistä</tspan>
    </text>
</svg>
    """
    filename = f"{temp_dir}/muted.png"
    cairosvg.svg2png(bytestring=svg.encode(), write_to=filename, output_width=width, output_height=height)
    return ImageClip(filename)

def make_header(year, start, end):
    svg = f"""<?xml version="1.0" encoding="UTF-8"?>
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="{width}" height="{height}">
    <text font-family="Russo One" font-size="60" fill="white" stroke="black" stroke-width="3" text-anchor="middle" x="50%" y="5%">
        <tspan x="50%" dy="1em">Suomen top-lista</tspan>
        <tspan x="50%" dy="1.1em" font-weight="bold">{year}</tspan>
        <tspan x="50%" dy="1.1em">Sijat {start} - {end}</tspan>
    </text>
</svg>
    """
    filename = f"{temp_dir}/header_{year}_{start}_{end}.png"
    cairosvg.svg2png(bytestring=svg.encode(), write_to=filename, output_width=width, output_height=height)
    return ImageClip(filename)

def make_cover(year, start, end):
    svg = f"""<?xml version="1.0" encoding="UTF-8"?>
<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="1080" height="1920">
  <polygon points="0,240 1000,240 1040,280 40,280 40,820 0,780" style="fill:#1fd5ee;" />
  <polygon points="1040,280 1040,820 40,820 80,860 1080,860 1080,320" style="fill:#fc2b55;" />
  <rect width="1000" height="540" style="fill:black;" x="40" y="280" />
  <text font-family="Russo One" font-size="104" fill="white" stroke="black" stroke-width="3" text-anchor="middle" x="50%" y="420">
    <tspan x="50%">Suomen top-lista</tspan>
    <tspan x="50%" dy="1.3em" font-weight="bold">{year}</tspan>
    <tspan x="50%" dy="1.3em">Sijat {start} - {end}</tspan>
  </text>
</svg>
    """
    filename = f"{temp_dir}/cover_{year}_{start}_{end}.png"
    cairosvg.svg2png(bytestring=svg.encode(), write_to=filename, output_width=width, output_height=height)
    return ImageClip(filename)

def make_song_title(year, ranking, artist, title):
    try:
        artist = html.escape(artist)
        title = html.escape(title)
        artist_size = 60
        title_size = 60
        
        if(len(artist) > 25):
            artist_size = 50
        if(len(artist) > 35):
            artist_size = 40

        if(len(title) > 25):
            title_size = 50
        if(len(title) > 35):
            title_size = 40

        svg = f"""<?xml version="1.0" encoding="UTF-8"?>
    <svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="{width}" height="{height}">
        <text font-family="Russo One" font-size="60" fill="white" stroke="black" stroke-width="3" text-anchor="middle" x="50%" y="84%">
            <tspan x="50%" dy="1em" font-weight="bold">{ranking}.</tspan>
            <tspan x="50%" dy="1em" font-size="{artist_size}">{artist}</tspan>
            <tspan x="50%" dy="1.1em" font-size="{title_size}">{title}</tspan>
        </text>
    </svg> 
        """
        filename = f"{temp_dir}/{year}_{ranking}.png"
        cairosvg.svg2png(bytestring=svg.encode(), write_to=filename, output_width=width, output_height=height)
        return ImageClip(filename)
    except:
        print(f'Error with {ranking} {artist} {title}')
        raise

def build_video(year, start, end):
    
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)

    songs = get_songs(year, start, end)
    songs = sorted(songs, key=lambda k: k['ranking'], reverse=True)

    

    # intro
    intro_bg = (ImageClip('cover3_2.png')
                .set_position(("center","center"))
                .set_duration(intro_length)
                .resize(width=width, height=height))
    intro_cover = (make_cover(year, start, end)
                   .set_position(("center","center"))
                   .set_duration(intro_length))
    intro_video = CompositeVideoClip([intro_bg, intro_cover], size=(width, height)).crossfadeout(1)

    clips = []

    for song in songs:

        length = video_length
        if song['audio_level'] == 0:
            length = muted_length

        print(f'Processing {song["ranking"]} {song["artist"]} {song["title"]} | {song["file"]} {song["start"]} -> {song["start"] + length}')

        if len(song['file']) == 0:
            video = make_missing().set_position(("center","center")).set_duration(3)
        else:
            video = VideoFileClip(song['file']).subclip(song['start'], song['start'] + length)
            video = video.set_position(("center","center"))
            video = video.audio_fadein(1).audio_fadeout(1)
            video = video.crossfadein(cross_fade)
            video = video.resize(height=1280)
            video = video.fx(afx.audio_normalize)
            if song['audio_level'] != 100:
                video = video.fx(afx.volumex, song['audio_level']/100)

        title = make_song_title(year, song['ranking'], song['artist'], song['title']).set_position(("center","bottom")).set_duration(video.duration)
        title = title.crossfadein(1).crossfadeout(1)

        if song['audio_level'] == 0:
            muted = make_muted().set_position(("center","center")).set_duration(length)
            video = video.fx(afx.volumex, 0)
            clip = CompositeVideoClip([video, title, muted], size=(width, height))
        else:
            clip = CompositeVideoClip([video, title], size=(width, height))
    
        clips.append(clip)


    songs_video = concatenate_videoclips(clips, method="compose")

    header = make_header(year, start, end).set_position(("center","top")).set_duration(songs_video.duration)
    songs_headers_video = CompositeVideoClip([songs_video, header], size=(width, height))

    final_clip = concatenate_videoclips([intro_video, songs_headers_video], method="compose")

    final_clip.write_videofile(f'{year} {start}-{end}.mp4', fps=30)


    for f in glob.glob(f"{temp_dir}/*"):
        os.remove(f)





build_video(1991, 46, 50)
"""
build_video(1991, 41, 45)
build_video(1991, 36, 40)
build_video(1991, 31, 35)
build_video(1991, 26, 30)
build_video(1991, 21, 25)
build_video(1991, 16, 20)
build_video(1991, 11, 15)
build_video(1991, 6, 10)
build_video(1991, 1, 5)

build_video(1991, 41, 50)
build_video(1991, 31, 40)
build_video(1991, 21, 30)
build_video(1991, 11, 20)
build_video(1991, 1, 10)
"""

Processing 50 R.E.M. SHINY HAPPY PEOPLE | 1991/50/REM - Shiny Happy People (1991) (HD) 4 720p.mp4 64.0 -> 84.0
Processing 49 KLF, The 3 AM ETERNAL | 1991/49/The KLF - 3 am Eternal (1991) HD 4K 2 720p.mp4 180.0 -> 200.0
Processing 48 Bad Boys Blue HOUSE OF SILENCE | 1991/48/Bad Boys Blue - House of Silence 4 720p.mp4 38.0 -> 58.0
Processing 47 Pedro's Heavy Gentlemen MILJOONAN MARKAN PAKARAT | 1991/47/Miljoonan markan pakarat 1 720p.mp4 24.0 -> 44.0
Processing 46 Neljä ruusua ELÄMÄ | 1991/46/Elämä 0 720p.mp4 88.0 -> 108.0
Moviepy - Building video 1991 46-50.mp4.
MoviePy - Writing audio in 1991 46-50TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video 1991 46-50.mp4



Moviepy - Done !
Moviepy - video ready 1991 46-50.mp4
Video 41 Klamydia HIHHULIT TULOO ep not found: Directory missing
Processing 45 Raptori TYYRIS TYLLERÖ | 1991/45/Raptori - Tyyris tyllerö (HQ) 1 720p.mp4 31.0 -> 51.0
Processing 44 Enigma MEA CULPA | 1991/44/Mea Culpa 2 720p.mp4 74.0 -> 94.0
Processing 43 Hector YHTENÄ ILTANA | 1991/43/Yhtenä iltana 1 720p.mp4 66.0 -> 86.0
Processing 42 Cooper, Alice HEY STOOPID | 1991/42/Hey Stoopid 0 360p.mp4 85.0 -> 105.0
Processing 41 Klamydia HIHHULIT TULOO ep |  nan -> nan
Moviepy - Building video 1991 41-45.mp4.
MoviePy - Writing audio in 1991 41-45TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video 1991 41-45.mp4



Moviepy - Done !
Moviepy - video ready 1991 41-45.mp4
Processing 40 Popeda KIRJE | 1991/40/Kirje (The Letter) 1 720p.mp4 63.0 -> 83.0
Processing 39 Hector JUODAAN VIINAA | 1991/39/Juodaan viinaa 2 720p.mp4 51.0 -> 71.0
Processing 38 Sorsakoski, Topi HAAVEKUVA | 1991/38/Haavekuva (Portrait Of My Love) 5 720p.mp4 55.0 -> 75.0
Processing 37 Popeda PUNAISTA JA MAKEAA | 1991/37/Punaista Ja Makeaa 2 720p.mp4 38.0 -> 58.0
Processing 36 Metallica THE UNFORGIVEN | 1991/36/Metallica - The Unforgiven (Official Music Video) 0 360p.mp4 83.0 -> 103.0
Moviepy - Building video 1991 36-40.mp4.
MoviePy - Writing audio in 1991 36-40TEMP_MPY_wvf_snd.mp3


...

Moviepy - Done !
Moviepy - video ready 1991 21-30.mp4
Processing 20 Isaak, Chris BLUE HOTEL | 1991/20/Blue Hotel 2 720p.mp4 71.0 -> 91.0
Processing 19 Kurre JÄIT SATEEN TAA | 1991/19/Jäit sateen taa 1 720p.mp4 48.0 -> 68.0
Processing 18 Eppu Normaali LENSIN MATALALLA 2 | 1991/18/Lensin Matalalla 2 0 720p.mp4 64.0 -> 84.0
Processing 17 Enigma SADENESS | 1991/17/Sadeness 0 360p.mp4 0.0 -> 20.0
Processing 16 Kolmas nainen ELÄMÄN TARKOITUS | 1991/16/Kolmas Nainen - Elämän tarkoitus (pitkä versio) 3 720p.mp4 154.0 -> 174.0
Processing 15 Isaak, Chris WICKED GAME | 1991/15/Wicked Game 1 720p.mp4 51.0 -> 71.0
Processing 14 Edelmann, Samuli PEGGY | 1991/14/Samuli Edelmann Peggy (live 1991) 1 360p.mp4 60.0 -> 80.0
Processing 13 Roxette FADING LIKE A FLOWER | 1991/13/Roxette - Fading Like A Flower (Every Time You Leave) (1991) HQ 3 720p.mp4 48.0 -> 68.0
Processing 12 Ne Luumäet ONNELLINEN PERHE | 1991/12/Onnellinen Perhe 1 720p.mp4 13.0 -> 33.0
Processing 11 Erasure LOVE TO HATE YOU | 1991/11/Lov

MoviePy - Done.
Moviepy - Writing video 1991 11-20.mp4



Moviepy - Done !
Moviepy - video ready 1991 11-20.mp4
Processing 10 Guns n' Roses DON'T CRY | 1991/10/Guns N Roses - Dont Cry  (Tokyo 1992)  HD Remastered 2 720p.mp4 0.0 -> 20.0
Processing 9 London Boys SWEET SOUL MUSIC | 1991/9/Sweet Soul Music 1 720p.mp4 15.0 -> 35.0
Processing 8 De La Soul RING RING RING (HA HA HEY) | 1991/8/Ring Ring Ring (Ha Ha Hey) (Official Music Video) [HD] 0 720p.mp4 0.0 -> 20.0
Processing 7 Hausmylly SE MUSTAMIES | 1991/7/Hausmylly - Se mustamies 2 360p.mp4 0.0 -> 20.0
Processing 6 Metallica ENTER SANDMAN | 1991/6/Enter Sandman 0 360p.mp4 0.0 -> 20.0
Processing 5 Inner Circle BAD BOYS | 1991/5/Inner Circle - Bad Boys (Official Music Video) 3 720p.mp4 0.0 -> 20.0
Processing 4 Guns n' Roses YOU COULD BE MINE | 1991/4/Guns N Roses - You Could Be Mine (Official Video HD) 0 720p.mp4 60.0 -> 80.0
Processing 3 Michael Jackson BLACK OR WHITE | 1991/3/Black or White (Official Video) 2 360p.mp4 152.0 -> 172.0
Processing 2 Roxette JOYRIDE | 1991/2/Joyride 1 720p.mp4 20.

MoviePy - Done.
Moviepy - Writing video 1991 1-10.mp4



Moviepy - Done !
Moviepy - video ready 1991 1-10.mp4
